In [1]:
# Download the Action Recommendation Dataset
!gdown "1ECcRj3u6g04z-4ODNSHoJC3jRz6ba0nd"

Downloading...
From (original): https://drive.google.com/uc?id=1ECcRj3u6g04z-4ODNSHoJC3jRz6ba0nd
From (redirected): https://drive.google.com/uc?id=1ECcRj3u6g04z-4ODNSHoJC3jRz6ba0nd&confirm=t&uuid=2d562d54-f13d-4b7f-b177-11583e02b6ea
To: /content/dataset.npy
100% 487M/487M [00:05<00:00, 82.1MB/s]


In [2]:
import numpy as np

with open("/content/dataset.npy", "rb") as f:
  dataset = np.load(f, allow_pickle=True)

len(dataset)

220065

In [3]:
cities = list(set([row['municipio'] for row in dataset]))
len(cities)

322

In [4]:
years = list(set([row['ano'] for row in dataset]))[2:]
print(f"{min(years)} até {max(years)}")

1948 até 2026


In [5]:
embeddings = [row['embedding'] for row in dataset]
embeddings = np.concat([embeddings])

embeddings.shape

(220065, 768)

In [6]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Carrega o modelo uma única vez (fora da função, para não recarregar toda hora)
model = SentenceTransformer("embaas/sentence-transformers-multilingual-e5-base")

def top_k_similares(texto: str,
                    embeddings: np.ndarray,
                    k: int = 5) -> np.ndarray:
    """
    Retorna os índices das k embeddings mais similares semanticamente ao `texto`.

    Parâmetros
    ----------
    texto : str
        Texto de entrada.
    embeddings : np.ndarray
        Matriz de embeddings com shape (N, 768), por exemplo (250000, 768).
        Idealmente pré-computadas com o mesmo modelo.
    k : int
        Número de vizinhos mais próximos a retornar.

    Retorno
    -------
    np.ndarray
        Array 1D com os índices das k embeddings mais similares.
    """

    # 1) Codificar o texto de entrada em um embedding
    # Para modelos E5, é comum prefixar com "query: "
    consulta = f"query: {texto}"
    embedding_texto = model.encode(
        consulta,
        normalize_embeddings=True  # já normaliza o vetor (norma L2 = 1)
    )  # shape (768,)

    # 2) Garantir que a matriz de embeddings também está normalizada
    #    (se você já salvou normalizada, pode pular esse passo)
    normas = np.linalg.norm(embeddings, axis=1, keepdims=True)
    # Evitar divisão por zero
    normas[normas == 0] = 1e-12
    embeddings_norm = embeddings / normas

    # 3) Similaridade por produto interno (equivale a cosseno se tudo está normalizado)
    #    resultado: vetor de similaridades shape (N,)
    scores = embeddings_norm @ embedding_texto

    # 4) Pegar os índices das k maiores similaridades
    # argpartition é mais eficiente que sort para top-k
    if k >= len(scores):
        # Se k >= N, só ordena tudo
        return np.argsort(-scores)

    idx_part = np.argpartition(-scores, k)[:k]  # k maiores (desordenados)
    # Ordenar esses k pelo score decrescente
    idx_ordenados = idx_part[np.argsort(-scores[idx_part])]

    return idx_ordenados


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
texto = "Como diminuir a criminalidade no município?"
k = 500

indices_top_k = top_k_similares(texto, embeddings, k)
print(indices_top_k)

[ 12479  48686   7426 213973  11019  94971 196688 210445 157034  37436
 168764 203921  97511  33654  14725  10419 158341 174584 118846 196125
 198567 197334 128310 213562 117036 108211 200695  56639 212763  46380
 148841  63776  12887  29797 164197   1106 130135 197511  20958  77571
 183625 150897 195491 130131 163983 144569 190195 148523 162955 117808
 141156 206339  46866  94430  53949  57498  72786 136196 186662  24626
  54428 138455 133062 214727   9560 134060   9979  13735 139995 175152
 124248 194842  58907 178995   1564 116079  12976  73205   7214  11644
 140249  69691   7278  32602 205807   7241 168737 165879  20921 173239
 125104 166680 163726 170183  20503  29792  21789  46394 171582 131656
 121916 127885  66888 109009  79770 201818  21753 179651  28725 152835
   6942 141249 150195 154028 114198  53935  67145 204112 181844  11632
  29769  38683 188439 148667 173212 191683  94575 176233  45878  34499
   9362 144850 190963 117232 199607 174615   9343 214475 167971 136423
  1122

In [13]:
results = dataset[indices_top_k]
[(row['municipio'], row['acao']) for row in results]

[('Caruaru',
  'Implementar programa de prevenção às drogas, denominado “Esporte Sim, Drogas Não”, no município.'),
 ('Guarujá do Sul', 'Criar o programa “Juro Zero” no município.'),
 ('Campina Grande',
  'Criar o programa municipal “Dignidade em Foco” no município.'),
 ('Natal', 'Criar o programa “Descarte Consciente” no município.'),
 ('Caruaru', 'Criar o programa “Mulheres Sem Violência” no município.'),
 ('Xangri-lá', 'Criar o programa “Deidentifique Nossas Ruas” no município.'),
 ('Sarandi', 'Criar o projeto “Diga Não às Drogas” no município.'),
 ('Pato Branco', 'Criar programa municipal antivandalismo no município.'),
 ('Itaporanga', 'Criar o programa “Infância Sem Violência” no município.'),
 ('Palmital', 'Criar o programa “Cidade Limpa” no município.'),
 ('Votorantim', 'Criar o projeto “Recorte Legal” no município.'),
 ('Natal',
  'Criar programa de cooperação e o código “Sinal Vermelho” no município para combater e prevenir a violência doméstica e familiar.'),
 ('Primavera do 

In [16]:
import numpy as np

emb = np.concat([row['embedding'][None] for row in results])
emb.shape

(500, 768)

In [26]:
import umap

umodel = umap.UMAP(
    n_neighbors=3,        # aumenta => tópicos mais globais
    n_components=2,       # 10–25 geralmente é bom
    min_dist=0.0,
    metric="cosine",
    random_state=42,
    verbose=True
)

umodel

UMAP(metric='cosine', min_dist=0.0, n_neighbors=3, random_state=42, verbose=True)

In [27]:
embeddings_umap = umodel.fit_transform(emb)
embeddings_umap.shape

/usr/local/lib/python3.12/dist-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


UMAP(angular_rp_forest=True, metric='cosine', min_dist=0.0, n_jobs=1, n_neighbors=3, random_state=42, verbose=True)
Tue Nov 18 15:59:24 2025 Construct fuzzy simplicial set
Tue Nov 18 15:59:25 2025 Finding Nearest Neighbors
Tue Nov 18 15:59:25 2025 Finished Nearest Neighbor Search
Tue Nov 18 15:59:25 2025 Construct embedding


Epochs completed:   0%|            0/500 [00:00]

	completed  0  /  500 epochs
	completed  50  /  500 epochs
	completed  100  /  500 epochs
	completed  150  /  500 epochs
	completed  200  /  500 epochs
	completed  250  /  500 epochs
	completed  300  /  500 epochs
	completed  350  /  500 epochs
	completed  400  /  500 epochs
	completed  450  /  500 epochs
Tue Nov 18 15:59:25 2025 Finished embedding


(500, 2)

In [28]:
import pandas as pd
import altair as alt

df = pd.DataFrame({
    "x": embeddings_umap[:, 0],
    "y": embeddings_umap[:, 1],
    "label": [row['acao'] for row in results],
})

alt.Chart(df).mark_circle().encode(
    x=alt.X("x:Q", axis=alt.Axis(title="UMAP-1")),
    y=alt.Y("y:Q", axis=alt.Axis(title="UMAP-2")),
    tooltip=[alt.Tooltip("label:N", title="Ação")]
).interactive()

alt.Chart(...)